In [198]:
import optuna
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
import sklearn
from math import sqrt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from optuna.integration import CatBoostPruningCallback
from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC

In [199]:
train = pd.read_csv('../data/data_real_train.csv', sep=';')

In [200]:
test = pd.read_csv('../data/data_real_test.csv', sep=';')

In [201]:
targets = ['class']
features2drop = []
filtered_features = [i for i in train.columns if (i not in targets and i not in features2drop)]

In [202]:
class_names = np.unique(train['class'])
train['class'] = train['class'].replace(class_names, np.arange(train['class'].nunique()))

In [203]:
X = train[filtered_features].drop(targets, axis=1, errors='ignore')
y = train['class']

In [204]:
X_test = test[filtered_features]

In [205]:
params_cat = {
             'n_estimators' : 200,
              # 'learning_rate': .03,
              'depth' : 3,
              'verbose': False,
              'use_best_model': True,
              'text_features': [],
              # 'train_dir' : '/home/jovyan/work/catboost',
              'border_count' : 64,
              'l2_leaf_reg' : 1,
              'bagging_temperature' : 2,
              'rsm' : 0.51,
              'loss_function': 'MultiClass',
              'auto_class_weights' : 'Balanced', #try not balanced
              'random_state': 42,
              'use_best_model': False,
              # 'custom_metric' : ['AUC', 'MAP'] # Не работает внутри sklearn.Pipelines
         }



cat_model = CatBoostClassifier(**params_cat)

In [206]:
params_cat_two = {
             'n_estimators' : 200,
              # 'learning_rate': .03,
              'depth' : 3,
              'verbose': False,
              'use_best_model': True,
              'text_features': [],
              # 'train_dir' : '/home/jovyan/work/catboost',
              'border_count' : 64,
              'l2_leaf_reg' : 1,
              'bagging_temperature' : 2,
              'rsm' : 0.51,
              'loss_function': 'MultiClass',
              'auto_class_weights' : 'Balanced', #try not balanced
              'random_state': 52,
              'use_best_model': False,
              # 'custom_metric' : ['AUC', 'MAP'] # Не работает внутри sklearn.Pipelines
         }



cat_model_two = CatBoostClassifier(**params_cat)

In [207]:
params_lgbm = {
    "num_leaves": 200,
    "n_estimators": 200,
    # "max_depth": 7,
    "min_child_samples": None,
    "learning_rate": 0.001,
    "min_data_in_leaf": 5,
    "feature_fraction": 0.98,
    # "categorical_feature": cat_cols,
    'reg_alpha' : 3.0,
    'reg_lambda' : 5.0,
}

lgbm_model = LGBMClassifier(**params_lgbm)

In [208]:
params_xgb = {
    "eta": 0.05,
    'n_estimators' : 200,
    "max_depth": 6,
    "subsample": 0.7,
    # "colsample_bytree": 0.95,
    'min_child_weight' : 0.1,
    'gamma': .01,
    'reg_lambda' : 0.1,
    'reg_alpha' : 0.5,
    "objective": "reg:linear",
    "eval_metric": "mae",
    'tree_method' : 'hist', # Supported tree methods for cat fs are `gpu_hist`, `approx`, and `hist`.
    'enable_categorical' : True
    
}

xgb_model = XGBClassifier(**params_xgb)

In [209]:
class Blender:
    def __init__(self, base_models, meta_model):
        self.base_models = base_models
        self.meta_model = meta_model

    def fit(self, X, y):
        n_fold = 3
        base_preds = []
        folds = KFold(n_splits=n_fold)
        
        for train_indices, val_indices in folds.split(X, y):
            X_train, X_val = X.loc[train_indices], X.loc[val_indices]
            y_train, y_val = y[train_indices], y[val_indices]
            models_preds = []
            
            for base_model in self.base_models:
                base_model.fit(X_train, y_train)
                models_preds.append(base_model.predict(X_val))
#                 print(y_val.values.reshape(-1, 1))
                
            base_preds.append(np.column_stack(models_preds))
            
        self.meta_model.fit(np.concatenate(base_preds), y)
            
        return base_preds

    def predict(self, X_test):
        base_preds = []
        
        for base_model in self.base_models:
            base_preds.append(base_model.predict(X_test))
        
        combined_preds = np.column_stack(base_preds)  
        preds = self.meta_model.predict(combined_preds)
        
        return preds

In [210]:
stacker = Blender(
    base_models=[cat_model, lgbm_model],
    meta_model=cat_model_two)

In [211]:
stacker.fit(X, y)

[LightGBM] [Warning] feature_fraction is set=0.98, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.98
[LightGBM] [Warning] feature_fraction is set=0.98, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.98
[LightGBM] [Warning] feature_fraction is set=0.98, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.98


[array([[1, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [0, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 0],
        

In [212]:
test_predict = pd.DataFrame(stacker.predict(X_test), columns=['class'])

In [213]:
''.join([str(i) for i in list(test_predict['class'])])

'01000001010000010110000110010100000001111001000111'

In [ ]:
01000000001000000100000001010000100001000000010001
01000001010000010110000110010100000001111001000111

In [214]:
test_predict['class'].value_counts()

0    32
1    18
Name: class, dtype: int64

In [215]:
test_predict['class'].value_counts()[1] / test_predict['class'].value_counts()[0]

0.5625

In [216]:
train['class'].value_counts()[1] / train['class'].value_counts()[0]

0.49700598802395207

In [217]:
train

,0,1,2,3,4,5,6,7,8,9,...,1491,1492,1493,1494,1495,1496,1497,1498,1499,class
0,2.546366,-1.060986,0.636591,-4.739070,-1.830741,-7.956681,-0.070732,-2.900028,-2.970760,-5.517126,...,2.192704,7.041310,4.083960,3.536619,6.295182,5.517126,2.617098,5.985114,3.802308,1
1,-2.859593,2.483331,10.384838,6.998478,10.862351,6.592323,-1.354544,1.429797,6.546963,3.837875,...,2.784341,-1.947732,1.198604,-12.868169,-4.364642,-2.182321,3.612118,-1.423343,2.472121,0
2,-2.305508,-0.720471,-0.216141,7.781090,1.147551,5.881197,-2.089367,-0.216141,-0.576377,8.069279,...,0.720471,0.788941,0.645497,5.403535,3.314168,-1.512990,1.657084,2.366823,2.079936,0
3,-3.401058,-3.060952,-0.612190,0.680212,-3.385706,-1.963710,-3.265016,-3.673143,-2.040635,0.612190,...,-2.448762,-0.406285,-2.369994,-13.808296,-4.761481,2.924910,-2.788868,1.421997,-2.234566,1
4,-6.665905,-6.319625,-2.423966,-7.618178,-7.325251,-8.790302,-6.839046,-7.358467,-3.549378,-9.436152,...,-2.770246,4.481330,1.378871,1.644834,5.367352,3.289668,-4.328510,2.757742,-0.947974,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-8.007427,-5.004642,-7.069057,-6.506035,-6.103013,-3.985641,-9.196030,-5.567664,-6.568593,-6.193245,...,4.066272,-2.740128,1.805994,-2.689995,0.312790,-4.003714,3.941156,-2.740128,2.117372,0
996,-9.994827,-5.879310,-2.832758,-0.267241,-2.181487,-1.064140,-8.391378,-4.543103,-2.458620,0.481034,...,2.725862,3.032800,3.192421,-0.053448,5.131034,0.267241,2.832758,1.862245,3.511663,0
997,3.391819,-1.413258,0.847955,1.695909,1.406879,3.564092,1.601692,-2.449647,1.601692,2.261213,...,4.333991,4.595803,4.595803,-7.066289,1.413258,1.507475,5.087728,4.970971,5.346138,0
998,2.616679,1.358660,-0.905773,0.251604,2.705055,1.602995,2.264434,0.754811,-2.415396,0.855453,...,-4.528867,-4.758892,-3.306178,-9.460300,-5.686244,-4.277263,-4.478546,-5.510296,-4.658705,1
